# Setup

In [ ]:
# !sudo apt-get install python3.5

In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyvi tqdm numpy gensim keras-tuner tensorflow pandas transformers simpletransformers pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from transformers import AutoModel, AutoTokenizer
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from simpletransformers.classification import ClassificationModel
import keras
from sklearn import svm, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Embedding, LSTM, CuDNNLSTM, Bidirectional, Dropout, Dense, Flatten, Activation, LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import Adam
from keras_tuner import Hyperband
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preprocessing

In [ ]:
path_data_train = '/content/drive/MyDrive/NCKH/CNN/data/data_train.txt'
path_data_original = '/content/drive/MyDrive/NCKH/CNN/data/data_original.xlsx'
path_data_original_cleaned = '/content/drive/MyDrive/NCKH/CNN/data/data_original_cleaned.xlsx'

# path_data_train = 'G:\\My Drive\\NCKH\\CNN\\data\\data_train.txt'
# path_data_original = 'G:\\My Drive\\NCKH\\CNN\\data\\data_original.xlsx'
# path_data_original_cleaned = 'G:\\My Drive\\NCKH\\CNN\\data\\data_original_cleaned.xlsx'

path_model_bin='/content/drive/MyDrive/NCKH/CNN/model/w2v.bin'
path_model_txt='/content/drive/MyDrive/NCKH/CNN/model/w2v.txt'

# path_model_bin='G:\\My Drive\\NCKH\\CNN\\model\\w2v.bin'
# path_model_txt='G:\\My Drive\\NCKH\\CNN\\model\\w2v.txt'

In [ ]:
def clean_text(text):
    # text = "".join(map(str, text))
    text = re.sub('<.*?>', '', text ).strip()
    text = re.sub('(\s)+', r'\1', text )
    return text

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

def sentence_segment(text):
    sents = re.split("([.?!])?[\n]+|[.?!] ", text)
    return sents

def word_segment(sent):
    sent = tokenize(sent)
    return sent

In [ ]:
# # list stopwords
# filename = '/content/drive/MyDrive/NCKH/CNN/data/stopwords.csv'
# data = pd.read_csv(filename, sep="\t", encoding='utf-8')
# list_stopwords = data['stopwords']

# def remove_stopword(text):
#     pre_text = []
#     words = text.split()
#     for word in words:
#         if word not in list_stopwords:
#             pre_text.append(word)
#     text2 = ' '.join(pre_text)

#     return text2

In [ ]:
# load data from xlsx file
data_original=pd.read_excel(path_data_original, header=0)

listsent=data_original['text'].values #load sentence
listlabel=data_original['label'].values # #load label

print("\nLoaded ",len(listsent),"sentences from data/data_original.xlsx.")
# print(listsent)

# for i in range(len(listsent)):
#     print(i, type(listsent[i]))


Loaded  3968 sentences from data/data_original.xlsx.


In [ ]:
#tokenize and clean text save to datatrain.txt
listsent2=[]
i=1
print("Preprocessing sentence:")
for sent in listsent:
    i=i+1
    sent = clean_text(sent)
    if(sent != None):
        sent = word_segment(sent)
        # sent = remove_stopword(normalize_text(sent))            
    listsent2=np.append(listsent2,sent)
    if(i%100==0):
        print("Processed sent:",i,"/",len(listsent))

print("Processed ",len(listsent),"sentences")

with open(path_data_train, 'w',encoding='utf-8') as f:
    for item in listsent2:
        f.write("%s\n" % item)
f.close()

print("\nWrite to file: data/data_train.txt")

df = pd.DataFrame({"text" : listsent2, "label" : listlabel})
df.to_csv(path_data_original_cleaned, encoding='utf-8',index=False)
print("Write to file: data/data_original_cleaned.csv")

Preprocessing sentence:
Processed sent: 100 / 3968
Processed sent: 200 / 3968
Processed sent: 300 / 3968
Processed sent: 400 / 3968
Processed sent: 500 / 3968
Processed sent: 600 / 3968
Processed sent: 700 / 3968
Processed sent: 800 / 3968
Processed sent: 900 / 3968
Processed sent: 1000 / 3968
Processed sent: 1100 / 3968
Processed sent: 1200 / 3968
Processed sent: 1300 / 3968
Processed sent: 1400 / 3968
Processed sent: 1500 / 3968
Processed sent: 1600 / 3968
Processed sent: 1700 / 3968
Processed sent: 1800 / 3968
Processed sent: 1900 / 3968
Processed sent: 2000 / 3968
Processed sent: 2100 / 3968
Processed sent: 2200 / 3968
Processed sent: 2300 / 3968
Processed sent: 2400 / 3968
Processed sent: 2500 / 3968
Processed sent: 2600 / 3968
Processed sent: 2700 / 3968
Processed sent: 2800 / 3968
Processed sent: 2900 / 3968
Processed sent: 3000 / 3968
Processed sent: 3100 / 3968
Processed sent: 3200 / 3968
Processed sent: 3300 / 3968
Processed sent: 3400 / 3968
Processed sent: 3500 / 3968
Proce

# BERT

In [ ]:
model_args = {
    "num_train_epochs": 5,
    "learning_rate": 1e-4,
    "overwrite_output_dir": True,
    "train_batch_size": 50,
    "use_early_stopping": True,
    "early_stopping_metric":	"eval_loss",
    "early_stopping_patience": 5,
}
model = ClassificationModel("roberta", "roberta-base", args = model_args)

df_train, df_test = train_test_split(df, test_size=0.1, shuffle = True)
df_train.columns = ["text", "labels"]
df_test.columns = ["text", "labels"]

display(df_train, df_test)

model.train_model(df_train, acc=accuracy_score)

result, model_outputs, wrong_predictions = model.eval_model(df_test, acc=accuracy_score)
result
# mcc - Matthews correlation coefficient
# tp - True positives
# tn - True negatives
# fp - False positives
# fn - False negatives
# auroc - Area Under Receiver Operating Characteristic curve
# auprc - Area Under Precision-Recall Curve
# eval_loss - Cross Entropy Loss for eval_df

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

,text,labels
922,khi k con vb,1
1973,mày biết tại_sao k thằng ngu vì đứng đấy ngăn_...,0
3851,Tuổi cũng cao rồi mụ già ơi . Sống sao cho con...,0
3739,Tình_hình dịch_bệnh đang rất phức_tạp . Mọi ng...,0
2189,Bộ mà ko dc phép của chính_phủ . Của Bộ chính_...,0
...,...,...
2452,Con mất day,0
210,Bua nao ve_sai gon gap em nha em .,1
2986,"Không thấy CS đâu hết , thấy có 3 con chó . Có...",0
184,Best_edit,1


,text,labels
1773,Trong đây ai chưa từng vượt đèn_đỏ dơ tay phát,1
3148,mấy cái thứ bắt_chước trộm hình của người khác...,0
521,Cưng muốn xĩu luôn á 33,1
1764,tream đi,1
923,"khi nào về thầy ơi , nhớ thầy quá rồi",1
...,...,...
2525,"Cùng ngành và bao_che cho nhau , sao biết nó l...",0
513,Cực_kì thuyết_phục 3,1
1250,nguyễn liêm nhầm_lẫn tai_hại,1
462,Cố lên cố lên,1


  0%|          | 0/3571 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/72 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

{'mcc': 0.5699100769986165,
 'tp': 158,
 'tn': 153,
 'fp': 34,
 'fn': 52,
 'auroc': 0.8569773363891011,
 'auprc': 0.8576352940180094,
 'acc': 0.783375314861461,
 'eval_loss': 0.5544063490629196}

In [ ]:
predictions, raw_outputs = model.predict([input()])
if predictions[0] == 0:
  print("This is a negative sentence!")
else:
  print("This is a positive sentence!")

Thằng mặt lồn


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

This is a negative sentence!


# Word2Vec

In [ ]:
sents = open(path_data_train, 'r',encoding='utf-8').readlines()
train_data = []
for sent in sents:
  train_data.append(sent.split())
train_data[0]

['cáp', 'đứt', 'tại', 'cá_mập', ',', 'gạch', 'vỡ', 'do', 'sinh_viên']

Trial and error to find the best parameters for the word2vec model

In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html
# IMPORTANT PARAMETER
# window (int, optional) – Maximum distance between the current and predicted word within a sentence.
# min_count (int, optional) – Ignores all words with total frequency lower than this.
# sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
# cbow_mean ({0, 1}, optional) – If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
# negative (int, optional) – Number of negative sampling will be used (noise words) (usually between 5-20).
word2vec_model = Word2Vec(train_data, vector_size=100, window=5, min_count=1, sample=0.001, workers=4, negative=20, sg=0, cbow_mean=0, epochs=50)

word2vec_model.wv.save_word2vec_format(path_model_bin, fvocab=None, binary=True)
word2vec_model.wv.save_word2vec_format(path_model_txt, fvocab=None, binary=False)

In [ ]:
# https://stackoverflow.com/questions/57149627/how-to-fix-the-input-function-in-python
# del input
print("Similar words:", word2vec_model.wv.most_similar(positive=input()))
print("Similar rate:", word2vec_model.wv.similarity(w1 = input("Word 1: "), w2 = input("Word 2: ")))

lồn
Similar words: [('xự', 0.9300951361656189), ('lol', 0.9298964738845825), ('loz', 0.9167914986610413), ('năng', 0.9163161516189575), ('l', 0.9143158793449402), ('lz', 0.9129595756530762), ('giáo_án', 0.9033104777336121), ('lon', 0.9026976823806763), ('lông', 0.9018619060516357), ('pede', 0.9005153775215149)]
Word 1: ngu
Word 2: dốt
Similar rate: 0.90541214


# Data embedding

In [ ]:
#PADDING AND EMBEDDING SENTENCES BY WORD EMBEDDING
def word_embed_sentences(model_type, model, sentences, labels, max_length=100):  
        embed_sentences = []
        embed_labels = []
        i = -1
        for sent in sentences:
            i = i + 1
            embed_sent = []
            sent=sent.split()
            
            if model_type == "phobert":
              for word in sent:
                embed_sent.append(model.encode(word))
              
            else:
              vocablist=model.key_to_index
              # print("Vocab size =", len(vocablist) + 1)
              for word in sent:
                # print("\nword=",word)
                if(word) not in vocablist:
                    print("Word '", word, "' not in vocabulary list!")
                else:
                    embed_sent.append(model[word])
      
            if len(embed_sent) < 1:
              continue
            elif len(embed_sent) > max_length:
                  embed_sent = embed_sent[:max_length] # if length of embedded sentence > max_length, get data in range 0:100
            elif len(embed_sent) < max_length:
                  embed_sent = np.concatenate((embed_sent, np.zeros(shape=(max_length - len(embed_sent),100), dtype=float)),axis=0) # if the length of the embedded sentence is < max_length, add zeros to the rest of the vector
            
            embed_sentences.append(embed_sent)
            embed_labels.append(labels[i])
            # print("Added",embed_sent,"to embed_sentences.")
        return embed_sentences, embed_labels

In [ ]:
data_original_cleaned=pd.read_csv(path_data_original_cleaned) #load data for train

listsent=data_original_cleaned['text'].values #load sentence
listlabel=data_original_cleaned['label'].values # #load label

print("Loaded",len(listsent),"sentences from data/data_goc_cleaned.csv.")

model = gensim.models.KeyedVectors.load_word2vec_format(path_model_bin, binary=True, encoding='utf-8')
# model = AutoTokenizer.from_pretrained("vinai/phobert-base")

sen_embedded, label_embedded = word_embed_sentences("word2vec", model, listsent, listlabel, max_length=100)

sen_embedded = np.array(sen_embedded)
label_embedded = np.array(label_embedded)

print("Embedded setences:",len(sen_embedded))
print("Embedded labels:", len(label_embedded))

Loaded 3968 sentences from data/data_goc_cleaned.csv.
Embedded setences: 3968
Embedded labels: 3968


# Build model

In [ ]:
print(device_lib.list_local_devices())
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2943218655577249690
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14343274496
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9887989169240491611
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(sen_embedded, label_embedded, test_size=0.1, shuffle = True)

n_train = int(0.9*len(sen_embedded))
n_test = int(0.1*len(sen_embedded) + 1)

x_train = x_train.reshape(n_train, -1)
x_test  = x_test.reshape(n_test, -1)
y_train = y_train.reshape(n_train, -1)
y_test = y_test.reshape(n_test, -1)

print("x_train shape:",x_train.shape)
print("y_train shape:",y_train.shape)
print("x_test shape:",x_test.shape)
print("y_test shape:",y_test.shape)

print("Example x_train: ",x_train[:2])
print("Example y_train: ",y_train[:2])
print("Example x_test: ",x_test[:2])
print("Example y_test: ",y_test[:2])

x_train shape: (3571, 10000)
y_train shape: (3571, 1)
x_test shape: (397, 10000)
y_test shape: (397, 1)
Example x_train:  [[-0.00480958  0.03818276  0.00508803 ...  0.          0.
   0.        ]
 [ 0.02010145  0.00417095  0.06119699 ...  0.          0.
   0.        ]]
Example y_train:  [[1]
 [0]]
Example x_test:  [[ 0.02826849  0.41419834  0.23217109 ...  0.          0.
   0.        ]
 [-0.00709484  0.40146041  0.36508414 ...  0.          0.
   0.        ]]
Example y_test:  [[1]
 [1]]


In [ ]:
# input_dim: input dimension = word_dim x max_length 
vocab_size = 50000
word_dim = 100
max_length = x_train.shape[1]
n_class =y_train.shape[1] #number of classes
n_epochs = 100
batch_size = 50
svm_model_path = "/content/drive/MyDrive/NCKH/CNN/model/{}.pickle"
cnn_model_path = "/content/drive/MyDrive/NCKH/CNN/model/{}.h5"

In [ ]:
def svm_model_builder():
  model = svm.SVC()
  
  return model

In [ ]:
def default_cnn_model_builder():
  model = Sequential()
  model.add(Embedding(vocab_size, word_dim, input_length=max_length)) # https://machinelearningcoban.com/tabml_book/ch_embedding/embedding.html
  # model.add(LSTM(50, return_sequences=True)) # For CPU
  # model.add(Bidirectional(LSTM(50, return_sequences=True))) # Best for CPU
  model.add(CuDNNLSTM(50, return_sequences=True)) # For GPU
  # model.add(Bidirectional(CuDNNLSTM(50, return_sequences=True))) # Best for GPU
  model.add(Dropout(0.2))
  model.add(Dense(25,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss=['binary_crossentropy'] , optimizer='adam', metrics=['accuracy'])
  model.summary()

  return model

In [ ]:
def tuned_cnn_model_builder(hp):
  hp_dims = hp.Int('input_dim', min_value=50, max_value=150, step=50)
  hp_units = hp.Int('units', min_value=25, max_value=50, step=25)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4])
  hp_dropout= hp.Choice('dropout_rate', values=[0.0, 0.1, 0.2])

  model = Sequential()
  model.add(Embedding(vocab_size, hp_dims, input_length=max_length)) # https://machinelearningcoban.com/tabml_book/ch_embedding/embedding.html
  # model.add(LSTM(hp_units, return_sequences=True)) # For CPU
  # model.add(Bidirectional(LSTM(hp_units, return_sequences=True))) # Best for CPU
  model.add(CuDNNLSTM(hp_units, return_sequences=True)) # For GPU
  # model.add(Bidirectional(CuDNNLSTM(hp_units, return_sequences=True))) # Best for GPU
  model.add(Dropout(hp_dropout))
  model.add(Dense(25,activation='relu'))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss=['binary_crossentropy'] , optimizer=Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])
  model.summary()
  
  return model

In [ ]:
es = EarlyStopping(monitor='val_accuracy', patience=10) # https://keras.io/api/callbacks/early_stopping/

method = int(input("1. Build model \n2. Load model  \nChoose method: "))
model_type = int(input("1. SVM model \n2. CNN model \nChoose model: "))

if method == 1:
  if model_type == 1:
    model = svm_model_builder()

  else:
    if input("Tune model (y/n): ").lower() == 'n':
      model = default_cnn_model_builder()

    else:
      # https://keras.io/api/keras_tuner/tuners/hyperband/
      tuner = Hyperband(tuned_cnn_model_builder,
                          objective='val_accuracy',
                          max_epochs=100,
                          directory='Tuner',
                          project_name='Negative Content Analysis - AutoTuner')
      tuner.search(x_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=[es])
      tuner.results_summary()
      best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
      model = tuner.hypermodel.build(best_hps)

else:
  name = input("Model name: ")

  if model_type == 1:
    model = pickle.load(open('/content/drive/MyDrive/NCKH/CNN/model/{}.pickle'.format(name), "rb"))
  else:
    model = keras.models.load_model('/content/drive/MyDrive/NCKH/CNN/model/{}.h5'.format(name))

if model_type == 1:
  history = model.fit(x_train, y_train.ravel())
  print("Test accuracy: ", model.score(x_test, y_test))

else:    
  history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=[es])

  loss, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
  print(f'Test loss: {loss}, test accuracy: {accuracy}')
  model.predict(input("Input a sentence: "))

Epoch 1/100
65/65 [==============================] - 34s 336ms/step - loss: 0.6025 - accuracy: 0.6523 - val_loss: 0.5848 - val_accuracy: 0.6536
Epoch 2/100
65/65 [==============================] - 19s 293ms/step - loss: 0.5978 - accuracy: 0.6583 - val_loss: 0.5821 - val_accuracy: 0.6592
Epoch 3/100
65/65 [==============================] - 19s 294ms/step - loss: 0.5903 - accuracy: 0.6536 - val_loss: 0.5869 - val_accuracy: 0.6592
Epoch 4/100
65/65 [==============================] - 19s 298ms/step - loss: 0.5857 - accuracy: 0.6642 - val_loss: 0.5875 - val_accuracy: 0.6648
Epoch 5/100
65/65 [==============================] - 19s 299ms/step - loss: 0.5825 - accuracy: 0.6608 - val_loss: 0.5878 - val_accuracy: 0.6620
Epoch 6/100
65/65 [==============================] - 24s 377ms/step - loss: 0.5818 - accuracy: 0.6623 - val_loss: 0.5955 - val_accuracy: 0.6536
Epoch 7/100
65/65 [==============================] - 19s 289ms/step - loss: 0.5779 - accuracy: 0.6704 - val_loss: 0.5974 - val_accuracy:

IndexError: ignored

In [ ]:
if input("Save model (y/n): ").lower() == "y":
  name = input("Model name: ")

  if model_type == 1:
    pickle.dump(model, open(svm_model_path.format(name), "wb"))
  else:
    model.save(cnn_model_path.format(name))

Save model (y/n): y
Model name: svm_model


Save model

In [ ]:
if model_type == 2:
  print(history.history.keys())

  plt.title('Model training and validation statistics')

  plt.subplot(1, 2, 1) # 1 rows, 2 cols, index 1
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.legend(['train accuracy', 'validation accuracy'], loc='best') # location
  plt.xlabel('epoch')
  plt.ylabel('accuracy')

  plt.subplot(1, 2, 2) # 1 rows, 2 cols, index 2
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.legend(['train loss', 'validation loss'], loc='best') # location
  plt.xlabel('epoch')
  plt.ylabel('loss')

  plt.subplots_adjust(wspace=0.5, hspace=0.5)

  plt.show()